In [26]:
import os, sys
import matplotlib.pyplot as plt
import numpy as np
from sklearn import decomposition, manifold

% matplotlib notebook

In [27]:
def compute_distance(x,y):
    x = x / np.linalg.norm(x)
    y = y / np.linalg.norm(y)
    return np.linalg.norm(x-y)

In [28]:
def print_percentage(n, t):
    sys.stdout.write('\r')
    sys.stdout.write("[%-20s] %d%%" % ('=' * ((n * 20/t) + 1) , n * 100/t + 1 ))
    sys.stdout.flush()

In [29]:
k = 400

In [30]:
feature_dir = '/fileserver/nmec-handwriting/localfeatures/nmec_bw_denoised_cc_deNNiam_fiel657_min500/'
#feature_dir = '/fileserver/nmec-handwriting/localfeatures/nmec_bw_cc_deNNiam_fiel657_min500'
#feature_dir = '/fileserver/nmec-handwriting/localfeatures/nmec_bw_crop_cc_deNNiam120_fiel657-120'

files = os.listdir(feature_dir)
files.sort()

feature_map = []
for i,filename in enumerate(files):
    path = feature_dir + "/" + filename
    #if os.path.isfile(path) and ( '008.' not in filename ):
    if os.path.isfile(path):
        try:
            feature_map.append( np.load(path).mean(axis=0) )
        except:
            feature_map.append( np.ones([1,4096]) * sys.maxint)
    print_percentage(i, len(files))
        
feature_map = np.array(feature_map)
feature_map.shape

[====================] 100%

(708, 4096)

In [31]:
pca = decomposition.PCA(n_components=k)
transform = pca.fit_transform(feature_map)
# transform = transform.T
transform.shape

(708, 400)

In [21]:
a = transform[0]
b = transform[1]
compute_min_distance(a,b)

NameError: name 'compute_min_distance' is not defined

In [32]:
transform = -feature_map

In [33]:
metric = []
for i, image in enumerate(transform):
    metricline = [np.array([compute_distance(image, other) for other in transform])]
    metric += metricline
    print_percentage(i, len(transform))

metric = np.array(metric)
F = -metric
np.fill_diagonal(F, -sys.maxint)

[====================] 100%

In [34]:
soft_correct = 0
hard_correct = 0
total_num = 0

k = 10
g = 8
max_top = 3

for j, i in enumerate(F):
    
    if j!=6:
        continue
    
    total_num += 1
    topk = i.argsort()[-k:]
    
    if files[j][:6] in (files[index][:6] for index in topk):
        soft_correct += 1
    
    hardsample = list(files[index][3:6] for index in topk[-max_top:])
    if len(set(hardsample)) == 1 and hardsample[0] == files[j][3:6]:
        print "%s matched %s" % (files[j][3:10], hardsample)
        hard_correct += 1

print "%-30s" % ( "-" * 37 )
print "SOFT CRITERIA: Top %d\t= %f" %(k, (soft_correct + 0.0) / total_num)
print "HARD CRITERIA: Top %d\t= %f" %(max_top, (hard_correct + 0.0) / total_num)

-------------------------------------
SOFT CRITERIA: Top 10	= 1.000000
HARD CRITERIA: Top 3	= 0.000000


In [35]:
F

array([[ -9.22337204e+18,  -3.91732898e-01,  -1.16587696e-01, ...,
         -3.17829227e-01,  -2.57119000e-01,  -3.29239565e-01],
       [ -3.91732898e-01,  -9.22337204e+18,  -3.55167754e-01, ...,
         -5.06661049e-01,  -4.95569678e-01,  -5.11344111e-01],
       [ -1.16587696e-01,  -3.55167754e-01,  -9.22337204e+18, ...,
         -3.49039958e-01,  -2.89078986e-01,  -3.63831805e-01],
       ..., 
       [ -3.17829227e-01,  -5.06661049e-01,  -3.49039958e-01, ...,
         -9.22337204e+18,  -1.75456085e-01,  -1.53746622e-01],
       [ -2.57119000e-01,  -4.95569678e-01,  -2.89078986e-01, ...,
         -1.75456085e-01,  -9.22337204e+18,  -1.48540848e-01],
       [ -3.29239565e-01,  -5.11344111e-01,  -3.63831805e-01, ...,
         -1.53746622e-01,  -1.48540848e-01,  -9.22337204e+18]])